In [47]:
import sqlite3
from sqlite3 import Error
import numpy as np
from pyteomics import pepxml

In [65]:
db_path = "/data/dominik/lower_order/pepxml_test.db"

conn = None
try:
    conn = sqlite3.connect(db_path, isolation_level=None)
    print(sqlite3.version)
except Error as e:
    print(e)


#conn.execute('pragma journal_mode=memory')
cursor = conn.cursor()

2.6.0


In [66]:
cursor.execute(create_spectra)
cursor.execute(create_results)

In [67]:
d = pepxml.read("/data/dominik/lower_order/Adult_Heart_bRP_Elite_39_f20.pep.xml")

In [68]:
k=0

m = 0
q = 0

temp_results=[(),]*100000
temp_item=[(),]*1000
        
        
for item in d:
    if m%1000 == 0: 
        
        print(f"{k}...", flush=False, end="")
        
        if m > 0:
            transaction_items(cursor, temp_item)
            #print(temp_results)
            transaction_results(cursor, temp_results)
            
        m=0
        q=0
        
        temp_results=[(),]*10000
        temp_item=[(),]*1000
        
    
    spec = item['spectrum']
    prec = item['precursor_neutral_mass']
    charge = item['assumed_charge']
    rt = item['retention_time_sec']
    scan = item['start_scan']

    item_pack = (scan, spec, prec, rt, charge)
    #print(m)
    temp_item[m] = item_pack
    m += 1
    
    #insert_spectra(cursor, item_pack)
                  
    for hit in item['search_hit']:
       
        pep = hit['peptide']
        hit_no = hit['hit_rank']
        
        scores = hit['search_score']
        
        deltacn = scores['deltacn']
        spscore = scores['spscore']
        sprank = scores['sprank']
        expect = scores['expect']
        xcorr = scores['xcorr']
        
        dm = hit['massdiff']
        
        hit_pack = (scan, hit_no, xcorr, deltacn, spscore, sprank,
                    expect, pep)
        
        #insert_results(cursor, hit_pack)
        
        temp_results[q] = hit_pack
        print(temp_results)
        q += 1

        
        
        
        

0...0...

IntegrityError: UNIQUE constraint failed: spectra.scan

In [51]:
def transaction_items(cursor, array):
    cursor.execute("BEGIN TRANSACTION")
    for chunk in range(len(array)):
        cursor.executemany(
            """insert into spectra(scan, spectrum, precursor_mass, rt_sec, charge) values (?, ?, ?, ?, ?);""",
            (array[chunk],)
            )
    cursor.execute("COMMIT")
    
    
def transaction_results(cursor, array):
    cursor.execute("BEGIN TRANSACTION")
    for chunk in range(len(array)):
        cursor.executemany(
            """insert into search_results(scan, hit, xcorr, deltacn, spscore, sprank, expect, peptide)
                VALUES(?,?,?,?,?,?,?,?);""",
            (array[chunk],)
            )
    cursor.execute("COMMIT")

In [79]:
next(d)['search_hit'][0]['search_score']

{'xcorr': 0.471,
 'deltacn': 0.161,
 'spscore': 3.8,
 'sprank': 5.0,
 'expect': 1.77}

In [ ]:
for path in paths:
            print(path)
            cur_file = pepxml.read(path)

            for spec in cur_file:
                if 'search_hit' in spec.keys():
                    if len(spec['search_hit']) == 10:
                        tev.append(list(map(lambda x: -0.02*np.log(spec['search_hit'][x]['search_score']['expect']/1000), np.arange(10))))
                        charges.append(int(spec['assumed_charge']))
                        big_n.append(int(spec['search_hit'][0]['num_matched_peptides']))

In [12]:
create_spectra = """CREATE TABLE IF NOT EXISTS spectra (
                    scan integer PRIMARY KEY,
                    spectrum text,
                    precursor_mass real,
                    rt_sec real,
                    charge integer
                    );"""
                    
create_results = """CREATE TABLE IF NOT EXISTS search_results (
                    id integer PRIMARY KEY,
                    hit integer,
                    xcorr float,
                    deltacn float,
                    spscore float,
                    sprank integer,
                    expect float,
                    peptide text,
                    scan integer,
                    FOREIGN KEY (scan) REFERENCES spectra (scan)
                    );"""

In [30]:
a = cursor.fetchall()

In [76]:
def insert_spectra(cursor, spectra):
    sql = """ INSERT INTO spectra(scan, spectrum, precursor_mass, rt_sec, charge)
                VALUES(?,?,?,?,?); """
    cursor.execute(sql, spectra)
    
    
def insert_results(cursor, spectra):
    sql = """ INSERT INTO search_results (scan, hit, xcorr, deltacn, spscore, sprank, expect, peptide)
                VALUES(?,?,?,?,?,?,?,?); """
    cursor.execute(sql, spectra)

In [26]:
insert_spectra(cursor, (120, "Hello world!", 123.14, 121234.44, 3))

In [64]:
conn.close()

In [ ]:
con = sqlite3.connect(self.target_db, isolation_level=None)
        cur = con.cursor()

        cur.execute("""create table if not exists likelihood (
                        id integer PRIMARY KEY,
                        pepid integer NOT NULL,
                        mzbin integer NOT NULL,
                        replicates integer NOT NULL,
                        mzp float NOT NULL,
                        mean float NOT NULL,
                        variance float NOT NULL
                   );""")
        
        
con.close()